### Prepare hydrologic model for sensitivity analysis ###
In control file, users need to provide a source hydrologic model. This source model works as a reference to build a new hydrologic model that is used for parameter estimation or sensitivity analysis. 
This code creates the new hydrologic model by:<br>
1. copying settings folder from source model to new model.
2. creating simulations folder in new model.
2. linking all other folders from source model to new model.

In [1]:
# import module
import os
import shutil
from datetime import datetime
import functions.utils as ut

In [2]:
# read paths from control_file
control_file = 'control_active.txt'
root_path = ut.read_from_control(control_file, 'root_path')
domain_name = ut.read_from_control(control_file, 'domain_name')
domain_path = os.path.join(root_path, domain_name)
model_src_path = ut.read_from_control(control_file, 'model_src_path')

In [3]:
# read new hydrologic model path
model_dst_path = ut.read_from_control(control_file, 'model_dst_path')
if model_dst_path == 'default':
    model_dst_path = os.path.join(domain_path, 'model')

#### 1. Copy settings folder from source model to new model. ####

In [4]:
scr_settings = os.path.join(model_src_path, 'settings')
dst_settings = os.path.join(model_dst_path, 'settings')
if os.path.exists(dst_settings):
    shutil.rmtree(dst_settings)
shutil.copytree(scr_settings, dst_settings, symlinks=False);

#### 2. Create simulations folder in new model ####

In [5]:
dst_simulations = os.path.join(model_dst_path, 'simulations')
if not os.path.exists(dst_simulations):
    os.makedirs(dst_simulations)

#### 3. Link all other folders from source model to new model. ####

In [6]:
# forcing, parameters, shapefiles folder
for folder in ['forcing', 'parameters', 'shapefiles']:
    scr = os.path.join(model_src_path, folder)
    dst = os.path.join(model_dst_path, folder)
    if not os.path.exists(dst):
        os.symlink(scr, dst)

#### 4. Update model paths in settings/fileManager.txt ####

In [7]:
summa_setting_path = os.path.join(model_dst_path, 'settings/SUMMA')
summa_filemanager = ut.read_from_control(control_file, 'summa_filemanager')
summa_filemanager_temp = summa_filemanager.split('.txt')[0]+'_temp.txt'

summa_filemanager = os.path.join(summa_setting_path, summa_filemanager)
summa_filemanager_temp = os.path.join(summa_setting_path, summa_filemanager_temp)

with open(summa_filemanager, 'r') as src:
    with open(summa_filemanager_temp, 'w') as dst:
        for line in src:
            if model_src_path in line:
                line = line.replace(model_src_path, model_dst_path)
            dst.write(line)
shutil.copy2(summa_filemanager_temp, summa_filemanager);
os.remove(summa_filemanager_temp);